<a href="https://colab.research.google.com/github/ammarsaf/aitinkerers-hackathon-supa-team-werecooked/blob/master/notebooks-benchmarking-exercises/benchmark_gemini_flash_llmasjudge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/refs/heads/master/datasets/for_presentation/boolq-eng-val-200.jsonl -q
!wget https://raw.githubusercontent.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/refs/heads/master/datasets/for_presentation/boolq-malay-val-200.jsonl -q
!wget https://raw.githubusercontent.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/refs/heads/master/datasets/for_presentation/fib-eng-val-200.jsonl -q
!wget https://raw.githubusercontent.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/refs/heads/master/datasets/for_presentation/fib-malay-val-200.jsonl -q

In [2]:
!pip install weave flash_attn accelerate bitsandbytes -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.8 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
GEMINI_APIKEY = userdata.get('GEMINI_APIKEY')
WEAVE_APIKEY = userdata.get("WEAVE_APIKEY")

# Test

In [4]:
import google.generativeai as genai
import os

In [5]:
genai.configure(api_key=GEMINI_APIKEY)

# model_gem = genai.GenerativeModel(model_name="gemini-1.5-flash")
# response = model_gem.generate_content("Hello gemini who are you?")
# print(response.text)

# Dependencies

In [6]:
from glob import glob
import json
from tqdm.notebook import tqdm
import time
import seaborn as sns
import pandas as pd
import weave
import re
from typing import Dict

# GPU agnostic

In [7]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


# Load dataset

In [8]:
dataset_list = glob('*.jsonl')
dataset_list

['fib-eng-val-200.jsonl',
 'boolq-eng-val-200.jsonl',
 'boolq-malay-val-200.jsonl',
 'fib-malay-val-200.jsonl']

In [9]:
# construct Malay + English dataset
data_all = []

for k in dataset_list:
    with open(k) as fopen:
        for d in tqdm(fopen):
            d = json.loads(d)
            data_all.append(d)

print(f'Size of dataset: {len(data_all)}')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Size of dataset: 800


In [10]:
data_all[0]

{'passage': 'Vehicles and pedestrians will now embark and disembark the Cowes ferry separately following Maritime and Coastguard Agency (MCA) guidance.\nIsle of Wight Council said its new procedures were in response to a resident\'s complaint.\nCouncillor Shirley Smart said it would "initially result in a slower service".\nOriginally passengers and vehicles boarded or disembarked the so called "floating bridge" at the same time.\nMs Smart, who is the executive member for economy and tourism, said the council already had measures in place to control how passengers and vehicles left or embarked the chain ferry "in a safe manner".\nHowever, it was "responding" to the MCA\'s recommendations "following this complaint".\nShe added: "This may initially result in a slower service while the measures are introduced and our customers get used to the changes."\nThe service has been in operation since 1859.',
 'question': "A new service on the Isle of Wight's chain ferry has been launched following

In [11]:
# construct Malay + English dataset
data_malay = []

for k in dataset_list:
    if 'malay' in k:
        with open(k) as fopen:
            for d in tqdm(fopen):
                d = json.loads(d)
                data_malay.append(d)

print(f'Size of dataset: {len(data_malay)}')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Size of dataset: 400


In [12]:
data_malay[0]

{'question': 'bolehkah anda memandu di kanada dengan lesen AS',
 'answer': 1,
 'passage': 'Orang yang memandu masuk ke Kanada mesti mempunyai dokumen pendaftaran kenderaan mereka dan bukti insurans.',
 'language': 'Malay'}

# Weave

In [13]:
PROJECT_NAME = 'benchmark_gemini-1.5-flash_llmasajudge_v1'

weave.init(PROJECT_NAME)

Please login to Weights & Biases (https://wandb.ai/) to continue:


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as Weights & Biases user: ammardev.
View Weave data at https://wandb.ai/ammardev/benchmark_gemini-1-5-flash_llmasajudge_v1/weave


In [ ]:
await evaluation_all.evaluate(geminiflash_asjudge)

# Benchmarking

In [14]:
def call_llm(message: str) -> str:
    """Function to call the LLM and generate output"""
    model_gem = genai.GenerativeModel(model_name="gemini-1.5-flash")
    safe = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
    ]
    response = model_gem.generate_content(message, safety_settings=safe)
    return response.text

In [15]:
# for test_data in data_all[:5]:
#     print(call_llm(prompt_v1.format(passage=test_data["passage"], question=test_data["question"])))

In [16]:
import time
import random
class GeminiFlashAsAJudge(weave.Model):
    prompt: str

    @weave.op
    def create_message(self, passage: str, question: str):
        return self.prompt.format(passage=passage, question=question)

    @weave.op
    def predict(self, passage:str, question:str):
        try:
            time.sleep(random.randint(3, 7))
            message = self.create_message(passage, question)
            return call_llm(message=message)
        except Exception as e1:
            print(f"Error e1: {e1}")
            print(f"Error -- question: {question}, passage: {passage}")
            pass

In [17]:
def accuracy(model_output, answer):
    try:
        try:
            model_output = json.loads(model_output)
            class_model_output = model_output.get('consistency', None)
        except json.JSONDecodeError:
            # to handle edge cases where the LLM outputs improper JSON like this: '1 {"consistency": 0'
            match = re.search(r'\"consistency\":\s*([01])', model_output)

            if match:
                number = match.group(1)
                class_model_output = int(number)
            else:
                class_model_output = None
        return {"accuracy": class_model_output == answer}
    except:
        pass


In [18]:
# for test_data in data_all[:5]:
#     model_output = call_llm(prompt_v1.format(passage=test_data["passage"], question=test_data["question"]))
#     # print(model_output)
#     acc = accuracy(model_output, test_data['answer'])
#     print(acc)

In [19]:

class BinaryMetrics(weave.Scorer):
    class_name: str
    eps: float = 1e-8

    @weave.op()
    def summarize(self, score_rows) -> dict:
        # filter out None rows, model may error out sometimes...
        score_rows = [score for score in score_rows if score["correct"] is not None]
        # Compute f1, precision, recall
        tp = sum([not score["negative"] and score["correct"] for score in score_rows])
        fp = sum([not score["negative"] and not score["correct"] for score in score_rows])
        fn = sum([score["negative"] and not score["correct"] for score in score_rows])
        precision = tp / (tp + fp + self.eps)
        recall = tp / (tp + fn + self.eps)
        f1 = 2 * precision * recall / (precision + recall + self.eps)
        result = {"f1": f1, "precision": precision, "recall": recall}
        return result

    @weave.op()
    def score(self, answer: dict, model_output: dict|str) -> dict:
        try:
            model_output = json.loads(model_output)
            class_model_output = model_output.get(self.class_name, None)
        except json.JSONDecodeError:
            # to handle edge cases where the LLM outputs improper JSON like this: '1 {"consistency": 1'
            match = re.search(r'\"consistency\":\s*([01])', model_output)

            if match:
                number = match.group(1)
                class_model_output = int(number)
            else:
                class_model_output = None
        result = {
            "correct": class_model_output == answer,
            "negative": not class_model_output,
        }
        return result

F1 = BinaryMetrics(class_name="consistency")

## Prompt 1

In [20]:
prompt_v1 = """Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca
dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana soalan/kenyataan yang
tidak disokong atau bercanggah dengan maklumat dalam dokumen).

### Anda perlu memilih antara dua pilihan berikut:
- Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.
- Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.

### Sebagai contoh:
Dokumen: "Gajah adalah mamalia besar yang biasanya ditemui di Afrika dan Asia. Mereka hidup dalam kumpulan yang dikenali sebagai kawanan dan terkenal kerana mempunyai ingatan yang baik."

Soalan/Kenyataan: "Gajah adalah mamalia besar yang biasanya ditemui di Eropah."
Jawapan: {{'consistency': 0}}

Soalan/Kenyataan: "Gajah adalah mamalia besar yang biasanya ditemui di Afrika dan Asia."
Jawapan: {{'consistency': 1}}

### Jawab berdasarkan dokumen dan soalan/kenyataan berikut:
Dokumen: {passage}
Soalan/Kenyataan: {question}

Kembalikan jawapan dalam format JSON untuk pilihan yang diberikan. Sebagai contoh: {{'consistency': 1}} atau {{'consistency': 0}}"""

In [21]:
geminiflash_asjudge = GeminiFlashAsAJudge(prompt=prompt_v1)

# Evaluate

## all data

In [22]:
!pip install --upgrade google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.6
    Uninstalling google-ai-generativelanguage-0.6.6:
      Successfully uninstalled google-ai-generativelanguage-0.6.6
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.7.2
    Uninstalling google-generativeai-0.7.2:
      Successfully uninstalled google-generativeai-0.7.2


In [23]:
evaluation_all = weave.Evaluation(dataset=data_all, scorers=[accuracy, F1])

In [24]:
await evaluation_all.evaluate(geminiflash_asjudge)

Evaluated 1 of 800 examples

Evaluated 2 of 800 examples

Evaluated 3 of 800 examples

Evaluated 4 of 800 examples

Evaluated 5 of 800 examples

...

Evaluated 798 of 800 examples

Evaluated 799 of 800 examples

Evaluated 800 of 800 examples

🍩 https://wandb.ai/ammardev/benchmark_gemini-1-5-flash_llmasajudge_v1/r/call/0192b24f-f091-7963-8b94-26ecb60b46fc


KeyError: 'correct'